In [82]:
from scipy import io
import numpy as np
import os
import tensorflow as tf
import tensorflow.keras.backend as K

from TA_CSPNN import *

In [67]:
for file in os.listdir('/Users/noahcarniglia/189project/matlab_data'):
    if not file == "A01T.mat":
        continue
    print(file)
    
    data = io.matlab.loadmat("matlab_data/" + file)['data']
    labels = io.matlab.loadmat('true_labels/' + file)['classlabel']
    break
print(data.shape)
print(labels.shape)

A01T.mat
(22, 250, 288)
(288, 1)


In [68]:
num_samples = data.shape[2]

train_size = int(num_samples * .9) # 90/10 train validation split
val_size = num_samples - train_size
print(f"num train samples: {train_size}")
print(f"num val samples: {val_size}")


num train samples: 259
num val samples: 29


In [70]:
rand_idx = np.random.randint(0,288, num_samples)
train_idx = rand_idx[:train_size]
val_idx = rand_idx[train_size:]

x_train = data[:,:,train_idx]
y_train = labels[train_idx]

x_val = data[:,:,val_idx]
y_val = labels[val_idx]
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")

x_train shape: (22, 250, 259)
y_train shape: (259, 1)
x_val shape: (22, 250, 29)
y_val shape: (29, 1)


In [74]:
x_train, y_train = K.constant(x_train), K.constant(y_train)
x_val, y_val = K.constant(x_val), K.constant(y_val)

In [80]:
x_train = tf.reshape(x_train, [259, 1, 22, 250])
x_val = tf.reshape(x_val, [29, 1, 22, 250])

In [81]:
print(f"x_train shape: {x_train.shape}")
print(f"x_val shape: {x_val.shape}")

x_train shape: (259, 1, 22, 250)
x_val shape: (29, 1, 22, 250)


In [52]:
class model_config(object):
    def __init__(self):
        self.channels = 22
        self.timesamples = 250
        self.timeKernelLen = 64
        self.num_classes = 4
        self.Ft = 8
        self.Fs = 2

In [53]:
config = model_config()
model = TA_CSPNN(config.num_classes, Channels=config.channels, Timesamples=config.timesamples,
                timeKernelLen = config.timeKernelLen, Ft=config.Ft, Fs=config.Fs)

In [63]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

In [64]:
model.compile(
    optimizer = Adam(),
    loss = CategoricalCrossentropy(from_logits=True)
)

In [ ]:
# input shape is (trials, 1, channels, time_samples)

In [83]:
history = model.fit(x_train, y_train, batch_size=64, epochs=5, validation_data=((x_val, y_val)))

Train on 259 samples, validate on 29 samples
Epoch 1/5
 64/259 [======>.......................] - ETA: 1s

InvalidArgumentError:  Default AvgPoolingOp only supports NHWC on device type CPU
	 [[node model_1/average_pooling2d_1/AvgPool (defined at <ipython-input-83-3300b8e19c8a>:1) ]] [Op:__inference_distributed_function_1582]

Errors may have originated from an input operation.
Input Source operations connected to node model_1/average_pooling2d_1/AvgPool:
 model_1/lambda_1/pow (defined at /Users/noahcarniglia/189project/TA_CSPNN.py:55)

Function call stack:
distributed_function
